In [1]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding1D, BatchNormalization, Flatten, Conv1D, AveragePooling1D, MaxPooling1D, GlobalMaxPooling2D, LeakyReLU, GlobalAveragePooling2D, ReLU, concatenate
from tensorflow.keras.initializers import glorot_uniform, constant
from tensorflow.keras.models import Model
import tensorflow as tf

import data
import numpy as np
import matplotlib.pyplot as plt
import filters
import json
import random


In [2]:
def resnet_block(X, f, s, filters, stage, block, dil=1):
   
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv1D(filters = F1, kernel_size = 1, strides = s, dilation_rate=dil, padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(momentum = 0.9, name = bn_name_base + '2a')(X)
    X = Activation(ReLU())(X)

    # Second component of main path (≈3 lines)
    X = Conv1D(filters = F2, kernel_size = f, strides = 1, dilation_rate=dil, padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(momentum=0.9, name=bn_name_base + '2b')(X)
    X = Activation(ReLU())(X)

    # Third component of main path (≈2 lines)
    X = Conv1D(filters = F3, kernel_size = 1, strides = 1, dilation_rate=dil, padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(momentum = 0.9, name = bn_name_base + '2c')(X)
    
    # side me
    X_shortcut = Conv1D(filters = F3, kernel_size = 1, strides = s, padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization( momentum = 0.9, name = bn_name_base + '1')(X_shortcut)
    
    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation(ReLU())(X)
    # X = BatchNormalization(momentum = 0.9, name = bn_name_base + '2c')(X)
    
    return X

In [ ]:
def ResNet50_1D(data_in_shape, num_output=2, fs=1000, UseDerivative=False):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER
    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes
    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(shape=data_in_shape)

    if UseDerivative:
        dt1 = (X_input[:,1:] - X_input[:,:-1])*fs
        dt2 = (dt1[:,1:] - dt1[:,:-1])*fs

        dt1 = tf.pad(dt1, tf.constant([[0,0],[0,1],[0,0]]))
        dt2 = tf.pad(dt2, tf.constant([[0,0],[0,2],[0,0]]))

        X = tf.concat([X_input, dt1, dt2], axis=2)
    else:
        X=X_input

    # Zero-Padding
    X = ZeroPadding1D(3)(X_input)

    # Stage 1
    X = Conv1D(64, 7, strides=2, name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=2, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = ZeroPadding1D(3)(X)
    X = MaxPooling1D(3, strides=3)(X)

    # Stage 2
    X = resnet_block(X, 3, 1, [64, 64, 256], stage=2, block='a')
    X = resnet_block(X, 3, 1, [64, 64, 256], stage=2, block='b')
    X = resnet_block(X, 3, 1, [64, 64, 256], stage=2, block='c')
    X = resnet_block(X, 3, 1, [64, 64, 256], stage=2, block='d')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = resnet_block(X, 3, 2, [128, 128, 512], stage=3, block='a')
    X = resnet_block(X, 3, 1, [128, 128, 512], stage=3, block='b')
    X = resnet_block(X, 3, 1, [128, 128, 512], stage=3, block='c')
    X = resnet_block(X, 3, 1, [128, 128, 512], stage=3, block='d')
    
    # Stage 4 (≈6 lines)
    X = resnet_block(X, 3, 2, [256, 256, 1024], stage=4, block='a')
    X = resnet_block(X, 3, 1, [256, 256, 1024], stage=4, block='b')
    X = resnet_block(X, 3, 1, [256, 256, 1024], stage=4, block='c')
    X = resnet_block(X, 3, 1, [256, 256, 1024], stage=4, block='d')
    
    # Stage 5 (≈3 lines)
    X = resnet_block(X, 3, 2, [512, 512, 2048], stage=5, block='a')
    X = resnet_block(X, 3, 1, [512, 512, 2048], stage=5, block='b')
    X = resnet_block(X, 3, 1, [512, 512, 2048], stage=5, block='c')
    X = resnet_block(X, 3, 2, [512, 512, 2048], stage=5, block='d')
    X = resnet_block(X, 3, 1, [512, 512, 2048], stage=5, block='e')
    
    
    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling1D(2, name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X_SBP = Dense(1, activation='linear', name='SBP', kernel_initializer = glorot_uniform(seed=0))(X)
    X_DBP = Dense(1, activation='linear', name='DBP', kernel_initializer = glorot_uniform(seed=0))(X)
    HR = Dense(1, activation='linear', name='HR', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    if num_output==3:
        model = Model(inputs=X_input, outputs=[X_SBP, X_DBP, HR], name='ResNet50_1D')
    else:
        model = Model(inputs = X_input, outputs = [X_SBP, X_DBP], name='ResNet50_1D')

    return model